# This is the section on data_cleansing process on second dataset.

In the beginning, let's import necessary library

In [1]:
#Import library
import pandas as pd
import numpy as np
import io
import requests
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

Then we start from import the second dataset, which describe the location where Residential buildings in which probable/confirmed cases have resided in the past 14 days or non-residential building with 2 or more probable/confirmed cases in the past 14 days.

Since the dataset is updated daily, we download the dataset on 2020-08-23 and save as building_list_eng_20200823.csv as our raw data.

In [7]:
#url="http://www.chp.gov.hk/files/misc/building_list_eng.csv"
ss=requests.get(url).content
covid_loc=pd.read_csv(io.StringIO(ss.decode('utf-8')))
covid_loc.to_csv('building_list_eng_20200823.csv') 
covid_loc

,District,Building name,Last date of residence of the case(s),Related probable/confirmed cases
0,Kwai Tsing,Ching Nga Court,NaN,3940
1,Sham Shui Po,"Tower 2, Heya Green",NaN,3941
2,Yau Tsim Mong,"Tai Ying Building, Cosmopolitan Estates",NaN,3942
3,Eastern,"Block 3, Fullview Garden",NaN,3943
4,Wong Tai Sin,"Lung Tai House, Lower Wong Tai Sin Estate",NaN,3946
...,...,...,...,...
1185,Central & Western,Wing On Centre (non-residential),21/08/2020,4636
1186,Kwai Tsing,CHT Tower (non-residential),21/08/2020,4638
1187,Eastern,114 Shau Kei Wan Road (non-residential),21/08/2020,"4638, 4647"
1188,Tsuen Wan,37 Ho Pui Street (non-residential),21/08/2020,4657


In this data, we can see the buildings where the cases resided before. But this data is not clean. Using building name and search directly will result error due to the followings:
1. Building name consist non address value (non-residential)
2. Building name contain Chinese Character
3. Building name is too specific (for example Wing Lok House, Fuk Loi Estate is too specific which is not found in geolocater. Instead Fuk Loi Estate is registered and can be found).

Hence, We will clean the building name before proceed to next step.

In [9]:
#copy the dataset to a new dataframe raw
raw = covid_loc
raw['Building name'] = raw['Building name'].str.replace('\(non-residential\)', '') # remove (non-residential)
raw['Building name'] = raw['Building name'].str.replace(r'[^\x00-\x7F]+', '') #remove chinese character

#Since there are rows having same location, we group them together
grouped_df = raw.groupby("Building name")
grouped_lists = grouped_df["Related probable/confirmed cases"].agg(lambda column: ", ".join(column))
grouped_lists = grouped_lists.reset_index(name="Related probable/confirmed cases")
grouped_lists

,Building name,Related probable/confirmed cases
0,1 Fu Chi Path,"1831, 2113, 3830"
1,1 Hung To Road No. 1 Hung To Canteen,"4186, 4387"
2,1 Wing Lung Street,4442
3,111 Tung Choi Street Fung Mei Yuen Restaurant,4654
4,114 Shau Kei Wan Road,"4638, 4647"
...,...,...
935,"Yuet Wing House, Tin Yuet Estate","4288, 4510"
936,"Yuk King House, King Ming Court","4297, 4298"
937,Zone B Tai Po Mega Mall,"2991, 3504"
938,iSQUARE,"3941, 4001"


We add one new column 'count' to indicate the number of confirmed cases in the buildings. Then, save the counted dataframe into csv file refined_20200819.csv.

In [11]:
# Use the 'Related probable/confirmed cases' column to count the number of cases occur in each building
def count(row):
    return row['Related probable/confirmed cases'].count(",") + 1

grouped_lists['count'] = grouped_lists.apply(count, axis=1)
 
grouped_lists.to_csv('refined_20200819.csv') 
grouped_lists

,Building name,Related probable/confirmed cases,count
0,1 Fu Chi Path,"1831, 2113, 3830",3
1,1 Hung To Road No. 1 Hung To Canteen,"4186, 4387",2
2,1 Wing Lung Street,4442,1
3,111 Tung Choi Street Fung Mei Yuen Restaurant,4654,1
4,114 Shau Kei Wan Road,"4638, 4647",2
...,...,...,...
935,"Yuet Wing House, Tin Yuet Estate","4288, 4510",2
936,"Yuk King House, King Ming Court","4297, 4298",2
937,Zone B Tai Po Mega Mall,"2991, 3504",2
938,iSQUARE,"3941, 4001",2


Now, we start to find out the coordinates of the builidng name/ location. First we input our foursquare credentials.

In [12]:
CLIENT_ID = '0LZUOF2I1OFCLMYS3YLAHQZ2XLY3FBGCX1HDJWGLP1DPFVAT' # your Foursquare ID
CLIENT_SECRET = 'DRT13X5NOEGWEGNSJ1XDGE1PCORCJ0STPQ5WP2I2UUYLASS3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0LZUOF2I1OFCLMYS3YLAHQZ2XLY3FBGCX1HDJWGLP1DPFVAT
CLIENT_SECRET:DRT13X5NOEGWEGNSJ1XDGE1PCORCJ0STPQ5WP2I2UUYLASS3


Now, we try to locate the coordinates of each building. Since there are some location name causing html connection request timeout, we separate dataset in smaller set and remove those location that caused error. So I decided to split the dataframe into five dataframes, and find out the latitude and longitude of each location for each datframe. In the end, combine it back into one.

In [14]:
grouped_lists = pd.read_csv(r'refined_20200819.csv')
grouped_lists_general = np.array_split(grouped_lists, 5)

grouped_lists_0 = grouped_lists_general[0]
grouped_lists_1 = grouped_lists_general[1]
grouped_lists_2 = grouped_lists_general[2]
grouped_lists_3 = grouped_lists_general[3]
grouped_lists_4 = grouped_lists_general[4]

#drop cases where building name that cause error in search below
#grouped_lists_2 = grouped_lists_2.drop(index=592)
grouped_lists_2

,Unnamed: 0,Building name,Related probable/confirmed cases,count
376,376,Kaiser Estate Phase I,4344,1
377,377,Kam Fai Building,"4328, 4479",2
378,378,"Kam Fai Building, Yee Kuk Street",4278,1
379,379,Kam Heng Building,4463,1
380,380,"Kam Ho Court, Treasure Bay","4119, 4318",2
...,...,...,...,...
559,559,Ocean Walk,"1797, 2153, 3690",3
560,560,"Oi Chak House, Oi Tung Estate",4437,1
561,561,"Oi Fai House, Yau Oi Estate","4006, 4007",2
562,562,Oi Man Market,"2900, 3082, 3575",3


Now we will perform the following code to add the coordinates to each grouped_list

In [16]:
#repeat the code by changing grouped_lists_0 to grouped_lists_1, then grouped_lists_2 until grouped_lists_4 in toappend.
toappend = grouped_lists_0

latitude_test =[]
longitude_test=[]
geolocator = Nominatim(user_agent="server_ltd")
for build in toappend['Building name']:
    address_test = build
    location = geolocator.geocode(address_test)
    if location == None:
        address_test = address_test.split(", ")
        location = geolocator.geocode(address_test[-1])
        if location == None:
                latitude = None
                longitude = None
        else:    
                latitude = location.latitude
                longitude = location.longitude
    else:
        latitude = location.latitude
        longitude = location.longitude
    latitude_test.append(latitude)
    longitude_test.append(longitude)

toappend['lat_col']=latitude_test
toappend['long_col']=longitude_test
toappend.to_csv('grouped_lists_20200823_0.csv') 
print("Completed")


KeyboardInterrupt: 

The error shown is because some building name is too general causing html timeout. For example you can see that the index value 592, or the building name for index 592 cannot be found using API, so the row is removed.
#grouped_lists_2 = grouped_lists_2.drop(index=592)

In general, all datasets are run and the location coordinates are save inside 5 datasets:
grouped_lists_0, 
grouped_lists_1, 
grouped_lists_2, 
grouped_lists_3, 
grouped_lists_4 